### Libraries

In [6]:
import pandas as pd
from IPython.core.display import display
from src.utils.preprocessing import preprocessing, standardize, dummy_code, oversample, remove_correlated_features
from src.utils.get_data import import_data
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
from sklearn.metrics import f1_score


ModuleNotFoundError: No module named 'src'

In [34]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [27]:
DATA_PATH = '../../data'
coarse_data, coarse_labels = import_data(DATA_PATH, segmentation_type = 'coarse', is_user_features=True)
fine_data, fine_labels = import_data(DATA_PATH, segmentation_type = 'fine', is_user_features=True)
no_data, no_labels = import_data(DATA_PATH, segmentation_type = 'no', is_user_features=True)

# For later processing rename the index
no_data.index = no_data.index.rename('subject')

### Preprocessing

In [11]:
def preprocessing(X, start = 0, stop = -3, thres = 0.95, norm = True, dummy = True, drop_corr = True):
    if norm:
        X = standardize(X, start, stop)
    if dummy:
        X = dummy_code(X, columns = ['Gender', 'Resp_Condition', 'Symptoms'])
    if drop_corr:
        X = remove_correlated_features(X, thres)
        
        return X

In [29]:
# rename
X_coarse = coarse_data
X_fine = fine_data
X_no = no_data

In [31]:
X_coarse = preprocessing(X_coarse)
X_fine = preprocessing(X_fine)
X_no = preprocessing(X_no)

### Grid search

In [33]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_coarse, labels=coarse_labels.Label, cv_k=5,
                                       params=KNN_PARAMS, metric=f1_score)

display(knn_results)

display(knn_results.iloc[knn_results[f1_score.__name__].agg(pd.Series.idxmax)])


KeyboardInterrupt: 